In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
df=pd.read_csv('../input/train.csv')
if(df.isna().any().any()):
    print("Yes")
df.head()

In [ ]:
y=df.iloc[:,0].values
X=df.drop(['label'],axis=1)
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X=scaler.fit_transform(X)
print(X[:2,:])

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=200)
X_copy=pca.fit_transform(X)
print(pca.explained_variance_ratio_)

In [ ]:
pca.explained_variance_ratio_.sum()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(n_jobs=-1)
rfc.fit(X_copy,y)
rfc.score(X_copy,y)

In [ ]:
from sklearn.model_selection import cross_val_score
accuracies=cross_val_score(estimator=rfc,X=X_copy,y=y,cv=10)
accuracies.mean()

In [ ]:
from sklearn.svm import SVC
svc=SVC()
svc.fit(X_copy[:7000,:],y[:7000])
from sklearn.model_selection import GridSearchCV
params=[{'C':[7,8],'kernel':['rbf'],'gamma':[0.002,0.001]}]
grid_search=GridSearchCV(estimator=svc,param_grid=params,scoring='accuracy',cv=5)
grid_search.fit(X_copy[:7000,:],y[:7000])
print(grid_search.best_score_)
print(grid_search.best_params_)

In [ ]:
svc=SVC(C=8,kernel='rbf',gamma=0.002)
svc.fit(X_copy[:20000,:],y[:20000])
accuracies=cross_val_score(estimator=svc,X=X_copy[:20000,:],y=y[:20000],cv=5)
accuracies.mean()

In [ ]:
svc=SVC(C=8,kernel='rbf',gamma=0.002)
svc.fit(X_copy,y)

In [ ]:
X_test=pd.read_csv('../input/test.csv')
X_test.head()

In [ ]:
X_test=scaler.transform(X_test)

In [ ]:
X_test=pca.transform(X_test)

In [ ]:
X_test.shape
len(X_test)
output=svc.predict(X_test)

In [ ]:
import csv
with open('./output.csv','w',newline='') as f:
    field=['ImageId','Label']
    writer=csv.DictWriter(f,fieldnames=field)
    writer.writeheader()
    for i in range(len(X_test)):
        writer.writerow({'ImageId':i+1,'Label':output[i]})

# This approach gives 0.97314 score on Kaggle 